<a href="https://colab.research.google.com/github/kojikokojiko/portfoilio/blob/master/M5/val_shop_id%E5%88%86%E3%81%91%2Bmean_enc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:


%cd /content/drive/My Drive/kaggle/M5/iwase

/content/drive/.shortcut-targets-by-id/18e73-MFsSq-bfNImjjh-axhWYXPQE-7c/kaggle/M5/iwase


In [0]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

from math import ceil

from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

In [0]:

BASE='../iwase_output/val/val_grid_part_1.pkl'
PRICE='../iwase_output/val/val_grid_part_2.pkl'
CALENDAR='../iwase_output/val/val_grid_part_3.pkl'


In [0]:
 df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)

In [0]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,wm_yr_wk,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique,price_momentum,price_momentum_m,price_momentum_y,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end
0,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12.0,0,11101,0.459961,0.500000,0.419922,0.019760,0.476318,0.919922,4.0,16,NaN,0.968750,0.949219,30,4,4,2,0,0,0,29,4,1,0,5,5,1
1,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2.0,0,11101,1.559570,1.769531,1.559570,0.032745,1.764648,0.881348,2.0,9,NaN,0.885742,0.896484,30,4,4,2,0,0,0,29,4,1,0,5,5,1
2,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,0,11101,3.169922,3.169922,2.970703,0.046356,2.980469,1.000000,2.0,20,NaN,1.064453,1.043945,30,4,4,2,0,0,0,29,4,1,0,5,5,1
3,HOBBIES_1_012_CA_1_validation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,0,11101,5.980469,6.519531,5.980469,0.115967,6.468750,0.916992,3.0,71,NaN,0.921875,0.958984,30,4,4,2,0,0,0,29,4,1,0,5,5,1
4,HOBBIES_1_015_CA_1_validation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,d_1,4.0,0,11101,0.700195,0.720215,0.680176,0.011337,0.706543,0.972168,3.0,16,NaN,0.990234,1.001953,30,4,4,2,0,0,0,29,4,1,0,5,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46881672,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,NaN,0,11617,2.980469,2.980469,2.480469,0.171631,2.800781,1.000000,5.0,206,1.0,1.029297,1.022461,30,4,4,2,0,0,0,22,20,5,5,4,6,1
46881673,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,NaN,0,11617,2.480469,2.679688,2.000000,0.253174,2.507812,0.925293,4.0,135,1.0,0.997070,1.112305,30,4,4,2,0,0,0,22,20,5,5,4,6,1
46881674,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,NaN,0,11617,3.980469,4.378906,3.980469,0.188599,4.117188,0.908691,3.0,150,1.0,0.965820,1.000000,30,4,4,2,0,0,0,22,20,5,5,4,6,1
46881675,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,NaN,230,11617,1.280273,1.280273,1.280273,0.000000,1.280273,1.000000,1.0,44,1.0,1.000000,1.000000,30,4,4,2,0,0,0,22,20,5,5,4,6,1


In [0]:


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [0]:
df=reduce_mem_usage(df)

Mem. usage decreased to 3175.88 Mb (36.0% reduction)


In [0]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,wm_yr_wk,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique,price_momentum,price_momentum_m,price_momentum_y,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end
0,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12.0,0,11101,0.459961,0.500000,0.419922,0.019760,0.476318,0.919922,4.0,16,NaN,0.968750,0.949219,30,4,4,2,0,0,0,29,4,1,0,5,5,1
1,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2.0,0,11101,1.559570,1.769531,1.559570,0.032745,1.764648,0.881348,2.0,9,NaN,0.885742,0.896484,30,4,4,2,0,0,0,29,4,1,0,5,5,1
2,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,0,11101,3.169922,3.169922,2.970703,0.046356,2.980469,1.000000,2.0,20,NaN,1.064453,1.043945,30,4,4,2,0,0,0,29,4,1,0,5,5,1
3,HOBBIES_1_012_CA_1_validation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,0,11101,5.980469,6.519531,5.980469,0.115967,6.468750,0.916992,3.0,71,NaN,0.921875,0.958984,30,4,4,2,0,0,0,29,4,1,0,5,5,1
4,HOBBIES_1_015_CA_1_validation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,d_1,4.0,0,11101,0.700195,0.720215,0.680176,0.011337,0.706543,0.972168,3.0,16,NaN,0.990234,1.001953,30,4,4,2,0,0,0,29,4,1,0,5,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46881672,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,NaN,0,11617,2.980469,2.980469,2.480469,0.171631,2.800781,1.000000,5.0,206,1.0,1.029297,1.022461,30,4,4,2,0,0,0,22,20,5,5,4,6,1
46881673,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,NaN,0,11617,2.480469,2.679688,2.000000,0.253174,2.507812,0.925293,4.0,135,1.0,0.997070,1.112305,30,4,4,2,0,0,0,22,20,5,5,4,6,1
46881674,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,NaN,0,11617,3.980469,4.378906,3.980469,0.188599,4.117188,0.908691,3.0,150,1.0,0.965820,1.000000,30,4,4,2,0,0,0,22,20,5,5,4,6,1
46881675,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,NaN,230,11617,1.280273,1.280273,1.280273,0.000000,1.280273,1.000000,1.0,44,1.0,1.000000,1.000000,30,4,4,2,0,0,0,22,20,5,5,4,6,1


In [0]:
STORE_IDS=list(df['store_id'].unique())
STORE_IDS

['CA_1',
 'CA_2',
 'CA_3',
 'CA_4',
 'TX_1',
 'TX_2',
 'TX_3',
 'WI_1',
 'WI_2',
 'WI_3']

In [0]:
# リセットインデックスはあえてしない
for store in STORE_IDS:
  temp_df=df[df["store_id"]==store]
  exec('df_'+store+'=temp_df')
  del temp_df
  gc.collect()
  exec("df_"+store+".to_pickle('../iwase_output/val/val_"+store+".pkl')")
  exec("del df_"+store)
  gc.collect()

  # exec('df_'+store+'=grid_df[grid_df["store_id"]=='+store+']')

In [0]:
# for store in STORE_IDS:
#   exec("df_"+store+".to_pickle('"+store+".pkl')")
#   exec("del df_"+store)
#   gc.collect()

In [0]:
STORE_IDS

['CA_1',
 'CA_2',
 'CA_3',
 'CA_4',
 'TX_1',
 'TX_2',
 'TX_3',
 'WI_1',
 'WI_2',
 'WI_3']

aggregeation特徴量をついかしわすれていたので、ついかしてpklファイル上書き

In [0]:
# STORE_IDS=['CA_1',
#  'CA_2',
#  'CA_3',
#  'CA_4',
#  'TX_1',
#  'TX_2',
#  'TX_3',
#  'WI_1',
#  'WI_2',
#  'WI_3']

In [0]:
icols =  [
            ['state_id'],
            ['store_id'],
            ['cat_id'],
            ['dept_id'],
            ['state_id', 'cat_id'],
            ['state_id', 'dept_id'],
            ['store_id', 'cat_id'],
            ['store_id', 'dept_id'],
            ['item_id'],
            ['item_id', 'state_id'],
            ['item_id', 'store_id']
            ]

In [0]:
TARGET = 'sales'  

In [0]:
for store in STORE_IDS:
  exec("grid_df=pd.read_pickle('../iwase_output/val/val_"+store+".pkl')")
  print("LOADED_"+store)
  for col in icols:
    print('Encoding', col)
    col_name = '_'+'_'.join(col)+'_'
    grid_df['enc'+col_name+'mean'] = grid_df.groupby(col)[TARGET].transform('mean').astype(np.float16)
    grid_df['enc'+col_name+'std'] = grid_df.groupby(col)[TARGET].transform('std').astype(np.float16)
  exec("grid_df.to_pickle('../iwase_output/val/val_"+store+".pkl')")
  print('completed_'+store)
  del grid_df
  gc.collect()


LOADED_CA_1
Encoding ['state_id']
Encoding ['store_id']
Encoding ['cat_id']
Encoding ['dept_id']
Encoding ['state_id', 'cat_id']
Encoding ['state_id', 'dept_id']
Encoding ['store_id', 'cat_id']
Encoding ['store_id', 'dept_id']
Encoding ['item_id']
Encoding ['item_id', 'state_id']
Encoding ['item_id', 'store_id']
completed_CA_1
LOADED_CA_2
Encoding ['state_id']
Encoding ['store_id']
Encoding ['cat_id']
Encoding ['dept_id']
Encoding ['state_id', 'cat_id']
Encoding ['state_id', 'dept_id']
Encoding ['store_id', 'cat_id']
Encoding ['store_id', 'dept_id']
Encoding ['item_id']
Encoding ['item_id', 'state_id']
Encoding ['item_id', 'store_id']
completed_CA_2
LOADED_CA_3
Encoding ['state_id']
Encoding ['store_id']
Encoding ['cat_id']
Encoding ['dept_id']
Encoding ['state_id', 'cat_id']
Encoding ['state_id', 'dept_id']
Encoding ['store_id', 'cat_id']
Encoding ['store_id', 'dept_id']
Encoding ['item_id']
Encoding ['item_id', 'state_id']
Encoding ['item_id', 'store_id']
completed_CA_3
LOADED_CA_4
E

In [0]:
pd.read_pickle('../iwase_output/val/val_WI_3.pkl')

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,wm_yr_wk,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique,price_momentum,price_momentum_m,price_momentum_y,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end,enc_state_id_mean,enc_state_id_std,enc_store_id_mean,enc_store_id_std,enc_cat_id_mean,enc_cat_id_std,enc_dept_id_mean,enc_dept_id_std,enc_state_id_cat_id_mean,enc_state_id_cat_id_std,enc_state_id_dept_id_mean,enc_state_id_dept_id_std,enc_store_id_cat_id_mean,enc_store_id_cat_id_std,enc_store_id_dept_id_mean,enc_store_id_dept_id_std,enc_item_id_mean,enc_item_id_std,enc_item_id_state_id_mean,enc_item_id_state_id_std,enc_item_id_store_id_mean,enc_item_id_store_id_std
9773,HOBBIES_1_004_WI_3_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,WI_3,WI,d_1,4.0,0,11101,4.339844,4.640625,4.339844,0.146118,4.523438,0.935547,2.0,8,NaN,0.959473,1.000000,30,4,4,2,0,0,0,29,4,1,0,5,5,1,1.371094,4.496094,1.371094,4.496094,0.535645,1.652344,0.651855,1.850586,NaN,NaN,NaN,NaN,0.535645,1.652344,0.651855,1.850586,2.708984,2.421875,NaN,NaN,2.708984,2.421875
9774,HOBBIES_1_008_WI_3_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,WI_3,WI,d_1,6.0,0,11101,0.459961,0.500000,0.419922,0.020035,0.475830,0.919922,4.0,16,NaN,0.968750,0.952637,30,4,4,2,0,0,0,29,4,1,0,5,5,1,1.371094,4.496094,1.371094,4.496094,0.535645,1.652344,0.651855,1.850586,NaN,NaN,NaN,NaN,0.535645,1.652344,0.651855,1.850586,2.468750,3.908203,NaN,NaN,2.468750,3.908203
9775,HOBBIES_1_009_WI_3_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,WI_3,WI,d_1,1.0,0,11101,1.559570,1.769531,1.559570,0.066345,1.720703,0.881348,3.0,9,NaN,0.900391,0.958008,30,4,4,2,0,0,0,29,4,1,0,5,5,1,1.371094,4.496094,1.371094,4.496094,0.535645,1.652344,0.651855,1.850586,NaN,NaN,NaN,NaN,0.535645,1.652344,0.651855,1.850586,0.717773,1.748047,0.725098,1.229492,0.717773,1.748047
9776,HOBBIES_1_010_WI_3_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,WI_3,WI,d_1,0.0,0,11101,3.169922,3.169922,2.970703,0.028915,2.974609,1.000000,2.0,16,NaN,1.064453,1.058594,30,4,4,2,0,0,0,29,4,1,0,5,5,1,1.371094,4.496094,1.371094,4.496094,0.535645,1.652344,0.651855,1.850586,NaN,NaN,NaN,NaN,0.535645,1.652344,0.651855,1.850586,0.784180,0.972168,NaN,NaN,0.784180,0.972168
9777,HOBBIES_1_012_WI_3_validation,HOBBIES_1_012,HOBBIES_1,HOBBIES,WI_3,WI,d_1,3.0,0,11101,5.980469,6.519531,5.980469,0.115967,6.468750,0.916992,3.0,70,NaN,0.921875,0.958984,30,4,4,2,0,0,0,29,4,1,0,5,5,1,1.371094,4.496094,1.371094,4.496094,0.535645,1.652344,0.651855,1.850586,NaN,NaN,NaN,NaN,0.535645,1.652344,0.651855,1.850586,0.468262,0.746582,0.182861,0.471191,0.468262,0.746582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46881672,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,NaN,0,11617,2.980469,2.980469,2.480469,0.171631,2.800781,1.000000,5.0,206,1.0,1.029297,1.022461,30,4,4,2,0,0,0,22,20,5,5,4,6,1,1.371094,4.496094,1.371094,4.496094,2.138672,6.148438,2.830078,7.710938,NaN,NaN,NaN,NaN,2.138672,6.148438,2.830078,7.710938,0.534180,1.176758,NaN,NaN,0.534180,1.176758
46881673,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,NaN,0,11617,2.480469,2.679688,2.000000,0.253174,2.507812,0.925293,4.0,135,1.0,0.997070,1.112305,30,4,4,2,0,0,0,22,20,5,5,4,6,1,1.371094,4.496094,1.371094,4.496094,2.138672,6.148438,2.830078,7.710938,NaN,NaN,NaN,NaN,2.138672,6.148438,2.830078,7.710938,0.376465,0.819824,0.295898,0.687500,0.376465,0.819824
46881674,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,NaN,0,11617,3.980469,4.378906,3.980469,0.188599,4.117188,0.908691,3.0,150,1.0,0.965820,1.000000,30,4,4,2,0,0,0,22,20,5,5,4,6,1,1.371094,4.496094,1.371094,4.496094,2.138672,6.148438,2.830078,7.710938,NaN,NaN,NaN,NaN,2.138672,6.148438,2.830078,7.710938,0.895020,1